<a href="https://www.kaggle.com/code/enricofindley/sms-text-classification-using-bidirectional-lstm?scriptVersionId=138328081" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing Libraries

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.7/489.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.4 MB/s eta 0:00:00


2023-07-30 04:57:58.604057: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-07-30 04:57:58.604131: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-07-30 04:57:58.604170: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Collection:
* **Collect data from api**
* **Save data from tsv to csv**

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"]).dropna()
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"]).dropna()

--2023-07-30 04:58:17--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2023-07-30 04:58:17 (9.14 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-07-30 04:58:18--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[============

# Data pre-processing:
* **Converting "type" column to numerical**
* **Creating Tensorflow dataset**
* **Preparing parameter and variable for training**

In [3]:
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]

In [4]:
train_labels =  train_df["type"].values
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df["msg"].values, train_labels)
)
test_labels =  test_df["type"].values
test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["msg"].values, test_labels)
)
train_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [5]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE) 
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Training, Validation, and Testing:
* **Creating model layer**
* **Predict 1 message as Testing**
* **Predict multiple messages as Testing**

In [6]:
vec = tf.keras.layers.TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vec.adapt(train_ds.map(lambda text, label: text))

In [7]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

In [8]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=10,
)

Epoch 1/10
131/131 [==============================] - 214s 2s/step - loss: 0.6106 - accuracy: 0.8660 - val_loss: 0.4960 - val_accuracy: 0.8604
Epoch 2/10
131/131 [==============================] - 192s 1s/step - loss: 0.3997 - accuracy: 0.8660 - val_loss: 0.2750 - val_accuracy: 0.8604
Epoch 3/10
131/131 [==============================] - 193s 1s/step - loss: 0.1704 - accuracy: 0.9232 - val_loss: 0.1034 - val_accuracy: 0.9667
Epoch 4/10
131/131 [==============================] - 195s 1s/step - loss: 0.0854 - accuracy: 0.9801 - val_loss: 0.0750 - val_accuracy: 0.9760
Epoch 5/10
131/131 [==============================] - 192s 1s/step - loss: 0.0614 - accuracy: 0.9861 - val_loss: 0.0671 - val_accuracy: 0.9771
Epoch 6/10
131/131 [==============================] - 192s 1s/step - loss: 0.0510 - accuracy: 0.9866 - val_loss: 0.0596 - val_accuracy: 0.9823
Epoch 7/10
131/131 [==============================] - 191s 1s/step - loss: 0.0428 - accuracy: 0.9890 - val_loss: 0.0529 - val_accuracy: 0.9854

In [9]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    ps = model.predict([pred_text])
    print(ps)
    p = ps[0][0]
    return [p, "ham" if p <0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 6s 6s/step
[[-6.039157]]
[-6.039157, 'ham']


In [10]:
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    correct_predictions = 0
    total_predictions = len(test_messages)

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] == ans:
            correct_predictions += 1

    accuracy_percentage = (correct_predictions / total_predictions) * 100
    print("Correct Prediction Percentage: {:.2f}%".format(accuracy_percentage))

test_predictions()

1/1 [==============================] - 0s 176ms/step
[[-6.039157]]
1/1 [==============================] - 0s 170ms/step
[[0.36745992]]
1/1 [==============================] - 0s 176ms/step
[[-12.517342]]
1/1 [==============================] - 0s 178ms/step
[[5.4527583]]
1/1 [==============================] - 0s 175ms/step
[[2.7908936]]
1/1 [==============================] - 0s 173ms/step
[[-8.998953]]
1/1 [==============================] - 0s 172ms/step
[[-11.550074]]
Correct Prediction Percentage: 85.71%
